## Using the Simulation Runner

In order to be able to run different simulations with different types of data (fairly) painlessly, it made sense to put together a simple API to run them together.

The goal was to try to do this as simply as possible while also being nice to use and easy to expand on. The general idea is this:

- Parameters that are used by different simulations/data generation are defined by defining them in json data to set them Runner object
    - For example: `{"num_samples": 20000, "num_labels": 3}`
- Simulations/data generation declare what parameters they need in order to be able to run. If any required parameters are missing in the json then the runner stops and prints out what parameters were missing.
- If all required parameters are set, then the data generation function runs and the data that is generated is passed to the simulation run function.

Creating the runner object immediately runs the simulation.

### Expanding the Runner

The runner should (hopefully) be easily expandable. When we end up needing different types of parameters, it should be very easy to define some more. For example, if we want to add a parameter called `delay`:
- Create a new parameter key in the `runner_keys.py` (`P_KEY_DELAY = "delay"`)
- Set this value in the parameter json for the simulators that need it
- Make sure any simulation/data generation that depends on it marks it as so (in `run_func_ltable` in `runner.py`)

Hopefully this isn't over-engineered. I tried very hard to keep this minimalistic

In [ ]:
from runner import Runner
import warnings

# hide the warning message temporarily
warnings.simplefilter("ignore")

# auto-reload the modules everytime a cell is run
%load_ext autoreload
%autoreload 2

### Overly simple json generator
Since the runner needs to be initialised with json parameter data, here's a simple function that converts a dictionary to a json string:

In [ ]:
import json

def gen_param_json_from_param_dict(param_dict):
    return json.dumps(param_dict)        

### Example: Running Federated Learning with Different Types of Data

Before we can run a simulation, we need to initialise the runner with the parameters that the simulation requires: 

In [ ]:
blob_data_file_path = "datasets/blob_S20000_L3_F4_U100.csv"

params_dict = {
    Runner.P_KEY_NUM_SAMPLES: 20000,
    Runner.P_KEY_NUM_LABELS: 3,
    Runner.P_KEY_NUM_FEATURES: 4,
    Runner.P_KEY_NUM_USERS: 100,
    Runner.P_KEY_NUM_ROUNDS: 10,
    Runner.P_KEY_BATCH_SIZE: 40,
    Runner.P_KEY_NUM_EPOCHS: 5,
    Runner.P_KEY_DATA_FILE_PATH: blob_data_file_path
}

json_params = gen_param_json_from_param_dict(params_dict)

With the JSON data, we're ready to run the simulation. Now we just need to construct the runner object and specify the data generation tpe

It's fine to run multiple simulations with a single runner object.

In [ ]:
_ = Runner(json_params, Runner.SIM_TYPE_FED_LEARNING, Runner.DATA_GEN_TYPE_DATA_FROM_FILE)

### Missing Parameters for a Run
If the simulation/data generation is missing any required parameters, you will get an exception like this:

In [ ]:
params_dict = {
    Runner.P_KEY_NUM_SAMPLES: 20000,
    Runner.P_KEY_NUM_LABELS: 3,
    Runner.P_KEY_NUM_FEATURES: 4,
    Runner.P_KEY_NUM_USERS: 100
}

json_params = gen_param_json_from_param_dict(params_dict)
_ = Runner(json_params, Runner.SIM_TYPE_FED_LEARNING, Runner.DATA_GEN_TYPE_DATA_FROM_FILE)